<a href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/SBG_autosegment_CT_Heart_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pyCERR DeepLab Heart Substurcture CT OAR Segmentation on Seven Bridges

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the Heart sub-structures on a lung CT scan using pyCERR.

## Software Requirements
* User account on CGC Seven Bridges and access to `aptea/pycerr-analyses` project
* Local python>=3.8

## Input Data Requirements
* RT planning DICOM CT
* RTSTRUCT with lung contour

### Running the model
The model is run as a task using preconfigured app on CGC Seven Bridges
```

### License
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.
    
This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.

You may publish papers and books using results produced using software provided that you reference the appropriate citations:
*  Heart sub-structures model: https://doi.org/10.1016/j.phro.2020.05.009
*  CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
*  CERR software: https://doi.org/10.1118/1.1568978
*  CERR radiomics: https://doi.org/10.1002/mp.13046


YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.



In [7]:
%%capture
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"
!pip install sevenbridges-python

In [2]:
import os
workDir = '/content'

In [3]:
#location of input DICOM folders
inputDicomPath = os.path.join(workDir,'input_dcm')
os.makedirs(inputDicomPath, exist_ok=True)

#location of output RTSTRUCT file
outputDicomPath = os.path.join(workDir, 'output_dcm')
os.makedirs(outputDicomPath, exist_ok=True)

#temp session directory
sessionPath = os.path.join(workDir, 'session_nii')
os.makedirs(sessionPath, exist_ok=True)

## Function Definitions: Data pre- and post-processing using pyCERR

### Preprocessing function: `processInputData`: Crop scan to LUNG extents

In [8]:
from cerr.dataclasses import structure as cerrStr
from cerr.contour import rasterseg as rs
from cerr.utils import mask
import numpy as np

def processInputData(scanNum, planC, lungNameList=['LUNG_TOTAL', 'LUNG_L', 'LUNG_R']):

    if isinstance(lungNameList, str):
        lungNameList = [lungNameList]

    # Extract scanArray
    scan3M = planC.scan[scanNum].getScanArray()
    mask3M = np.zeros(scan3M.shape, dtype=bool)

    # List of Structure names
    strNames = [s.structureName for s in planC.structure]
    numOrigStructs = len(strNames)

    # Get total lung mask
    for lungName in lungNameList:
        lungInd = cerrStr.getMatchingIndex(lungName.upper(), strNames, 'exact')
        if len(lungInd) > 0:
            # Get lung extents
            mask3M = mask3M | rs.getStrMask(lungInd[0], planC)

    if not np.any(mask3M):
        raise Exception('Lung contour name did not match any structures in planC')

    # Create cropped scan
    rmin,rmax,cmin,cmax,smin,smax,_ = mask.computeBoundingBox(mask3M)
    x,y,z = planC.scan[0].getScanXYZVals()
    xCropV = x[cmin:cmax]
    yCropV = y[rmin:rmax]
    zCropV = z[smin:smax]
    scan3M = planC.scan[0].getScanArray()
    scanCrop3M = scan3M[rmin:rmax,cmin:cmax,smin:smax]

    return scanCrop3M, (xCropV, yCropV, zCropV)


### Postprocessing function: `postProcAndImportSeg`: Import AI segmentations to planC (retain only the largest connected component for each structure)

In [9]:
#Import label map to CERR
from glob import glob
from cerr import plan_container as pc

atriaLabelDict = {1: 'DL_Atria'}
heartSubSegDict = {2: 'DL_AORTA', 3: 'DL_LA',
                   4: 'DL_LV', 5: 'DL_RA',
                   6: 'DL_RV', 7: 'DL_IVC',
                   8: 'DL_SVC', 9: 'DL_PA'}
heartSegDict = {1: 'DL_heart'}
periLabelDict = {1: 'DL_Pericardium'}
ventriLabelDict = {1: 'DL_Ventricles'}

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
    niiGlob = glob(os.path.join(outputDir,'*.nii.gz'))
    for segFile in niiGlob:
        print('Importing ' + segFile + '...')
        # Get segFile name
        if 'heart.nii.gz' in segFile:
            strToLabelMap = heartSubSegDict
        elif 'heartStructure.nii.gz' in segFile:
            strToLabelMap = heartSegDict
        elif 'atria.nii.gz' in segFile:
            strToLabelMap = atriaLabelDict
        elif 'pericardium.nii.gz' in segFile:
            strToLabelMap = periLabelDict
        elif 'ventricles.nii.gz' in segFile:
            strToLabelMap = ventriLabelDict
        numLabel = len(strToLabelMap)
        numStrOrig = len(planC.structure)
        planC = pc.loadNiiStructure(segFile, scanNum, planC, \
                                  labels_dict = strToLabelMap)
        numStructs = len(planC.structure)
        cpyStrNumV = np.arange(numStrOrig,numStructs)
        numConnComponents = 1
        for structNum in cpyStrNumV:
            _, planC = cerrStr.getLargestConnComps(structNum, numConnComponents, planC, \
                                            saveFlag=True, replaceFlag=True)

    return planC

In [10]:
#%%capture
import subprocess
import cerr
from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.ai_pipeline import createSessionDir, getScanNumFromIdentifier
from cerr.dcm_export import rtstruct_iod

modality = 'CT SCAN'
lungNameList = ['LUNG_CNTR', 'LUNG_IPSI'] #List lung contour name(s)

folderList = glob(os.path.join(inputDicomPath,'*'))
# Example for one
dcmDir = folderList[0]
print(dcmDir)

### Preprocess DICOM in pyCERR and run segmentation on Seven Bridges: Generate OARs for the CT scan located at `dcmDir`

In [13]:
fname = os.path.basename(dcmDir)
# Create session dir to store temporary data
modInputPath, modOutputPath = createSessionDir(sessionPath, dcmDir)

# Import DICOM to planC
planC = pc.loadDcmDir(dcmDir)

# Identify scan index in  planC
scanIdS = {"imageType": modality}
matchScanV = getScanNumFromIdentifier(scanIdS, planC, False)
scanNum = matchScanV[0]

# Pre-process data
procScan3M, resizeGridS = processInputData(scanNum, planC, lungNameList)
planC = pc.importScanArray(procScan3M, resizeGridS[0], \
        resizeGridS[1], resizeGridS[2], modality, scanNum, planC)
procScanNum = len(planC.scan) - 1

# Export inputs to NIfTI
scanFilename = os.path.join(modInputPath, f"{fname}_scan_3D.nii.gz")
planC.scan[procScanNum].saveNii(scanFilename)

numOrigStructs = len(planC.structure)

('0617-305105', '0617-305105', '1.3.6.1.4.1.14519.5.2.1.6329.6468.145697334858590437505784366193', '1.3.6.1.4.1.14519.5.2.1.6329.6468.123197441458886075851466636569', 'CT', '', '', '', '', '', '', '')
('0617-305105', '0617-305105', '1.3.6.1.4.1.14519.5.2.1.6329.6468.145697334858590437505784366193', '1.3.6.1.4.1.14519.5.2.1.6329.6468.330216875011191373593880879988', 'RTSTRUCT', '', '', '', '', '', '', '')


In [43]:
import sevenbridges as sbg
from getpass import getpass
from sevenbridges.errors import SbgError
url = 'https://cgc-api.sbgenomics.com/v2'
proj = 'aptea/pycerr-analyses'

In [ ]:
#User needs to be a member of our aptea/pycerr-analyses project on 7 Bridges
#Token can be found under the Developer menu
authtoken = getpass()

In [15]:
api = sbg.Api(url=url, token=authtoken)
sbproj = api.projects.get(proj)

In [16]:
#upload the preprocessed input NIfTI to 7 Bridges filesystem
api.files.upload(scanFilename, project=sbproj)

In [55]:
#get the file identifier of the uploaded input NIfTI
fbase = os.path.basename(scanFilename).split('.')[0]
sbg_nii_file = [f for f in api.files.query(project=proj) if fbase in f.name][0]

In [ ]:
#create unique timestamp for task ID
from datetime import datetime
dateobj = datetime.now()
timestamp = '_'.join((str(dateobj.year), str(dateobj.month), str(dateobj.day), str(dateobj.hour), str(dateobj.minute), str(dateobj.microsecond)))
print(timestamp)

In [ ]:
# Assign a task ID

task_name = '_'.join(('CT_Heart_OAR_DataStudio_',fbase,timestamp))
print('Task name ' + task_name)

# App I want to use to run a task
app = proj + '/ct-heart-oar-setup-container-cli/29'

# Inputs
inputs = {}
inputs['input_nifti'] = sbg_nii_file

try:
    task = api.tasks.create(name=task_name, project=proj, app=app, inputs=inputs, run=True)
except SbgError as e:
    print('I was unable to run the task.')
    print(e.message)

In [58]:
#create folder on 7 Bridges to store task output
task_output_storage_folder_id = '67116c87c4827b6d9b78c68f' #do not modify this, it is the folder location on the 7 Bridges file system
sbg_session_folder = api.files.create_folder(
    name=task_name, parent=task_output_storage_folder_id
)

In [63]:
#after task completes (email notification), download task output to local folder and move output on 7 Bridges to the storage folder
task_nii_files = [f for f in api.files.query(project=proj) if fbase in f.name]
for f in task_nii_files:
    print(f.name)
    if f.name != os.path.basename(scanFilename):
      f.download(path=os.path.join(modOutputPath,f.name))
    f.move_to_folder(parent=sbg_session_folder)

In [ ]:
#import results into planC
planC = postProcAndImportSeg(modOutputPath, procScanNum, scanNum, planC)
numStructs = len(planC.structure)

In [ ]:
# Export segmentations to DICOM
structFileName = fname + '_AI_seg_RTSTRUCT.dcm'
structFilePath = os.path.join(outputDicomPath,structFileName)
structNumV = np.arange(numOrigStructs, numStructs)
indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID, planC) for structNum in structNumV], dtype=int)
origIndsToExportV = structNumV[indOrigV == scanNum]
seriesDescription = "pyCERR CT_Heart_OAR AI-Generated"
exportOpts = {'seriesDescription': seriesDescription}
rtstruct_iod.create(origIndsToExportV,structFilePath,planC,exportOpts)

# Visualize results for the last CT scan using Napari Viewer

## Display using `matplotlib`

In [62]:
from cerr.viewer import showMplNb
showMplNb(planC=planC, scan_nums=scanNum,
          struct_nums=origIndsToExportV,
          windowCenter=-400, windowWidth=2000)

Output()